In [87]:
import json
import sys
import os
import shutil
import math
import numpy as np
import cv2
import pandas as pd
import csv
from PIL import Image,ImageDraw,ImageFont,ImageOps
#from imutils import perspective
from scipy.spatial import distance as dist
import numpy as np
import cv2
import ast

In [88]:
# 先跑 Validation
Is_train=False

In [89]:
# 先跑 Train
Is_train=True

In [90]:
if Is_train: 
    img_path='/workspace/text_spot_rec/data/train_label.txt'
if not Is_train:    
    img_path='/workspace/text_spot_rec/data/test_label.txt'

In [91]:
def order_points(pts):
	# initialize a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype="float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis=1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis=1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype="float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [92]:
result=[]
dic={}
with open(os.path.join(img_path), 'r') as f:
        for i in f:
            img=i.split('\t')[0].split('/')[-1]
            tmp=i.split('\t')[-1]
            tmp=tmp[1:len(tmp)-1]
            #print(tmp)
            if tmp[-2:]=='}]':
                 #tmp=tmp[0:len(tmp)-2] 
                tmp=tmp.replace("}]","")
                #print(tmp)
            tmp_list=tmp.split('},')
            #print(tmp_list)
            for j in range(len(tmp_list)):
                tmp_str=tmp_list[j].strip().replace('\\\\"','')
                #print(tmp_str)
                if tmp_str[-1]==']':
                    tmp_str=tmp_str+'}'
                #print(tmp_str)
                dic=ast.literal_eval(tmp_str)
                result.append([img,dic['points'][0][0],dic['points'][0][1],
                               dic['points'][1][0],dic['points'][1][1],
                               dic['points'][2][0],dic['points'][2][1],
                               dic['points'][3][0],dic['points'][3][1],
                               dic['transcription']])    

In [93]:
df_tmp=pd.DataFrame(result)
df_tmp.columns=['img','X0','Y0','X1','Y1','X2','Y2','X3','Y3','label']

In [71]:
if Is_train: 
    df_tmp.to_csv('data/Task3_train_label.csv',index=False)
if not Is_train: 
    df_tmp.to_csv('data/Task3_val_label.csv',index=False)

In [72]:
def cropImg(imgName, imgPath, points,output_img_path,newImgName):
  #  img = cv2.imread('./test_data/public/public/img_public/img_6000.jpg')
    img = cv2.imread(imgPath+imgName)

    #cropped = perspective.four_point_transform(img, points[0])
    cropped = four_point_transform(img, points[0]) 
    cv2.imwrite(output_img_path+newImgName, cropped)
    
    w=cropped.shape[1]
    h=cropped.shape[0]
    if w*1.4 < h:
        #print (width,height)
        cropped = cv2.rotate(cropped, cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.imwrite(output_img_path+newImgName, cropped)    

In [73]:
# Training dataset
train_data_path = '/workspace/text_spot_rec/data/'
train_img_src_path = '/workspace/text_spot_rec/data/train/img/'
train_img_out_path = '/workspace/text_spot_rec/data/train_crop/'

In [74]:
# Validation dataset
val_data_path = '/workspace/text_spot_rec/data/'
val_img_src_path = '/workspace/text_spot_rec/data/train/img/'
val_img_out_path = '/workspace/text_spot_rec/data/val_crop/'

In [75]:
# Private dataset
private_data_path = '/workspace/text_spot_rec/data/'
private_img_src_path = '/workspace/text_spot_rec/data/private/'
private_img_out_path = '/workspace/text_spot_rec/data/private_crop/'

In [76]:
# Public dataset
public_data_path = '/workspace/text_spot_rec/data/'
public_img_src_path = '/workspace/text_spot_rec/data/public/'
public_img_out_path = '/workspace/text_spot_rec/data/public_crop/'

In [102]:
if Is_train: 
     df_label = pd.read_csv(train_data_path+'Task3_train_label.csv')
if not Is_train:
     df_label = pd.read_csv(train_data_path+'Task3_val_label.csv')

In [105]:
df_label.Y0=df_label.Y0.astype(np.int64)
df_label.Y1=df_label.Y1.astype(np.int64)
df_label.Y2=df_label.Y2.astype(np.int64)
df_label.Y3=df_label.Y3.astype(np.int64)

In [106]:
df_label.to_csv('data/Task3_train_label.csv',index=False)

In [78]:
#Private
#df_label = pd.read_csv(private_data_path+'Task3_private_label.csv')

In [79]:
#Public
#df_label = pd.read_csv(public_data_path+'Task3_public_label.csv')

In [80]:
df_label['points']=''

In [81]:
df_list =df_label.to_numpy()

In [82]:
df_list

array([['img_913.jpg', 610, 701, ..., 832, '日月光國際飯店', ''],
       ['img_913.jpg', 94, 748, ..., 863, 'HOTEL', ''],
       ['img_3295.jpg', 143, 623, ..., 696, '貓咪貓奴', ''],
       ...,
       ['img_10836.jpg', 377, 894, ..., 942, 'UNOCHA', ''],
       ['img_10836.jpg', 312, 961, ..., 989, 'TAIWAN', ''],
       ['img_10836.jpg', 624, 969, ..., 997, 'TEA', '']], dtype=object)

In [83]:
length = len(df_list)
for i in range(length):
    df_list[i][10]= np.asarray([[[df_list[i][1],df_list[i][2]],
                                [df_list[i][3],df_list[i][4]],
                                [df_list[i][5],df_list[i][6]],
                                [df_list[i][7],df_list[i][8]]]])

In [84]:
len(df_list)

11689

In [85]:
def getNewImgName(Strcount):
    while len(Strcount)<5:
        Strcount='0'+Strcount
    return Strcount

In [86]:
def cropSourceImg(img_src_path,img_out_path):
    data_counter = 1  
    for img in df_list:
        tmp = img_src_path + img[0]
        #print(tmp)
        cropImg(img[0], img_src_path, img[10], img_out_path, getNewImgName(str(data_counter))+'_'+img[0])
        data_counter = data_counter + 1
    print('finished !')


In [22]:
if Is_train:
    cropSourceImg(train_img_src_path,train_img_out_path)
if not Is_train:
    cropSourceImg(val_img_src_path,val_img_out_path)

finished !


In [1]:
#cropSourceImg(public_img_src_path,public_img_out_path)

In [2]:
#cropSourceImg(private_img_src_path,private_img_out_path)